In [1]:
def get_opening(i, df):
    """ Returns opening capacity """

    if(i != 1):
        return df.at[i-1, 'closing']

def get_power(i, df):
    """ Calculates and returns raw power """
    
    power = 0
    if(df.at[i,'charge_forecast'] == 1):
        power = -min(BATTERY_POWER, (BATTERY_CAPACITY-df.at[i,'opening'])/EFFICIENCY*2)

    elif(df.at[i,'discharge_forecast'] == 1):
        power = min(BATTERY_POWER, df.at[i,'opening']/EFFICIENCY*2)
        
    return power

def get_dispatch(i, df):
    """ Calculates and returns market dispatch """
    
    rawPower = df.at[i, 'raw_power']
    
    if(rawPower < 0):
        dispatch = (rawPower/2)

    else:
        # EFFICIENCY is already in decimal (0.9) so no need to divide by 100
        dispatch = (rawPower/2 * EFFICIENCY)

    return dispatch

def get_closing(i, df):
    """ Calculates and returns closing capacity"""
    
    dispatch = df.at[i, 'dispatch']
    opening_cap = df.at[i, 'opening']
    
    if(dispatch < 0):
        condition = opening_cap - (dispatch * EFFICIENCY)
    else:
        condition = opening_cap - (dispatch * (100/(EFFICIENCY*100)))
            
    return round(max(0, min(condition, BATTERY_CAPACITY)),0)

def get_revenue(i, df):
    """ Calculates and returns revenue"""
    
    price = df.at[i, PRICE]
    dispatch = df.at[i, 'dispatch']
    
    if(dispatch < 0):
        revenue = round(dispatch * price * (1/MARGINAL_LOSS_FACTOR),0)
    else:
        revenue = round(dispatch * price * MARGINAL_LOSS_FACTOR,0)
    
    return revenue

In [ ]:
def calculate(df):
    
    for i in list(df.index):
        
        # update opening capacity
        df.at[i, 'opening'] = get_opening(i, df)

        # finding raw_power
        df.at[i, "raw_power"] = get_power(i, df)

        # finding market_dispatch
        df.at[i, "dispatch"] = get_dispatch(i, df)

        # finding closing_capacity
        df.at[i,'closing'] = get_closing(i, df)

        #finding revenue
        df.at[i,'revenue'] = get_revenue(i, df)
            
    return df

In [ ]:
def algorithm2(ori_df):
    """ Finds optimal charge and discharge period from the mean """

    summer_df = ori_df[(ori_df['month'] == 1) | (ori_df['month'] == 2) | (ori_df['month'] == 12)]
    autumn_df = ori_df[(ori_df['month'] == 3) | (ori_df['month'] == 4) | (ori_df['month'] == 5)]
    winter_df = ori_df[(ori_df['month'] == 6) | (ori_df['month'] == 7) | (ori_df['month'] == 8)]
    spring_df = ori_df[(ori_df['month'] == 9) | (ori_df['month'] == 10) | (ori_df['month'] == 11)]
    
    seasons_df = [summer_df, autumn_df, winter_df, spring_df]
    for season in seasons_df:
        spot_price = ori_df.groupby(['Period'])[[PRICE]].mean()

        # First cycle
        min_index, max_index = first_cycle(spot_price)

        # Second cycle
        # Comment line 11 if only want one cycle
        min_index, max_index = sec_cycle(spot_price, min_index, max_index)
        
        print(min_index, max_index)
        
    # the charge and discharge period are fixed in Algorithm 2, +1 to get their periods
    charge_period = list(np.asarray(min_index) + 1)
    discharge_period = list(np.asarray(max_index) + 1)
    
    return charge_period, discharge_period